### CNN 이미지 분류 모델
- 이미지 데이터셋 준비
    - torhchvision의 내장 데이터셋 활용 CIFIA10
- 이미지 분류 모델
    - 커스텀 CNN 구현

[1] 모듈 로딩 및 데이터 준비 <hr>

In [17]:
import torch
import torch.nn
from torchvision.datasets import CIFAR10        # torchvison 내장데이터셋
from torchvision.transforms import ToTensor     # torchvision 데이터 변환 관련 모듈
from torch.utils.data import DataLoader         # 데이터셋 관련 모듈 로딩
import matplotlib.pyplot as plt

In [18]:
## 데이터 로딩
DIR_PATH='../data/'

# Pytorch의 Dataset 형태 로딩
cifarDS=CIFAR10(DIR_PATH,
                train=True,
                download=False,
                transform=ToTensor())

In [19]:
type(cifarDS)

torchvision.datasets.cifar.CIFAR10

In [20]:
# Dataset의 속성 확인
print(f'classes : {cifarDS.classes}')
print(F' class to idx : {cifarDS.class_to_idx}')
print(F' shape : {cifarDS.data.shape}')
print(F' targets length : {len(cifarDS.targets)}')

classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
 class to idx : {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
 shape : (50000, 32, 32, 3)
 targets length : 50000


[2] 데이터 로더 <hr>
- 학습시 배치크기만큼 데이터와 라벨/타겟을 로딩

In [21]:
BATCH_SIZE=50

cifar10DL=DataLoader(cifarDS,batch_size=5)

In [22]:
for data,target in cifar10DL:
    print(data.shape)
    print(target)
    #print(data.numpy())
    # plt.imshow(data.squeeze().transpose(0,2))
    # plt.title(cifarDS.classes[target.item()])
    break

""" 
데이터 로더 사용시 채널이 앞으로 옴!
data. numpy() < array로 바꾸고
그림을 그리고 싶다면 젤 앞에 있는 1을 지우고(squeeze)
채널을 뒤로 밀어야함 (data.T)
"""

torch.Size([5, 3, 32, 32])
tensor([6, 9, 9, 4, 1])


' \n데이터 로더 사용시 채널이 앞으로 옴!\ndata. numpy() < array로 바꾸고\n그림을 그리고 싶다면 젤 앞에 있는 1을 지우고(squeeze)\n채널을 뒤로 밀어야함 (data.T)\n'

[3] 커스텀 모델 설계 및 정의 <hr>
- 모델목적 : 이미지 분류 모델
- 학습방법 : 지도학습 > 분류 > 다중분류 (10개)
- 클래스이름 : ImageMCF
- 클래스구조 : 특징추출부분 => CNN + 학습부분 FC
- 부모클래스 : nn.Module

In [23]:
import torch.nn as nn
import torch.nn.functional as F

In [24]:
class ImageMCF(nn.Module):
    # 모델구조 설계 즉, 생성자 메서드
    def __init__(self):
        # 부모 생성
        super().__init__()
        # 모델 층 구성
        # 특징 추출 층
        self.cnn_layer=nn.Sequential(
            nn.Conv2d(3,10,kernel_size=3),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            nn.MaxPool2d(2))

        # [1, 3, 32, 32] : input
        self.in_layer=nn.Conv2d(3,10,3)
        # [1, 10, 30, 30]
        self.p_layer=nn.MaxPool2d(2)
        # [1, 10, 15, 15]

        # 학습 관련 층
        self.hd_layer=nn.Linear(10*15*15 ,20)
        self.out_layer=nn.Linear(20 ,10)

    # 전방향/순방향 학습 메서드
    def forward(self,input):
        # 이미지 틍징 맵 추출
        output=self.in_layer(input)
        print(f'[output1] {output.shape}')

        output=F.relu(output)
        print(f'[output2] {output.shape}')

        output=self.p_layer(output)
        print(f'[output3] {output.shape}')

        # 4D -> 2D (샘플수, 피처수)
        output=output.view(output.shape[0],-1)
        print(f'[output4] {output.shape}')

        output=F.relu(self.hd_layer(output))
        print(f'[output5] {output.shape}')

        output=self.out_layer(output)
        print(f'[output6] {output.shape}')

        return output

In [29]:
class ImageMCF(nn.Module):
    # 모델구조 설계 즉, 생성자 메서드
    def __init__(self):
        # 부모 생성
        super().__init__()
        # 모델 층 구성
        # 특징 추출 층
        self.cnn_layer=nn.Sequential(
            nn.Conv2d(3,10,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.cnn_layer=nn.Sequential(
            nn.Conv2d(10,30,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2))
        

        # 학습 관련 층
        self.hd_layer=nn.Linear(30*7,7 ,20)
        self.out_layer=nn.Linear(20 ,10)

    # 전방향/순방향 학습 메서드
    def forward(self,input):
        # 이미지 틍징 맵 추출
        output=self.cnn_layer(input)

        # 4D -> 2D (샘플수, 피처수)
        output=output.view(output.shape[0],-1)
        print(f'[output4] {output.shape}')

        output=F.relu(self.hd_layer(output))
        print(f'[output5] {output.shape}')

        output=self.out_layer(output)
        print(f'[output6] {output.shape}')

        return output

In [25]:
from torchinfo import summary
m=ImageMCF()
summary(m)

Layer (type:depth-idx)                   Param #
ImageMCF                                 --
├─Conv2d: 1-1                            280
├─MaxPool2d: 1-2                         --
├─Linear: 1-3                            45,020
├─Linear: 1-4                            210
Total params: 45,510
Trainable params: 45,510
Non-trainable params: 0

In [28]:
for data,target in cifar10DL:
    print(f'[target]==> {target}')
    print(data.shape)
    # 학습
    pre=m(data)
    print(pre)
    print(f'pre => {pre.argmax(dim=1)}')
    break

[target]==> tensor([6, 9, 9, 4, 1])
torch.Size([5, 3, 32, 32])
[output1] torch.Size([5, 10, 30, 30])
[output2] torch.Size([5, 10, 30, 30])
[output3] torch.Size([5, 10, 15, 15])
[output4] torch.Size([5, 2250])
[output5] torch.Size([5, 20])
[output6] torch.Size([5, 10])
tensor([[-0.0138,  0.1802, -0.2684,  0.0279, -0.2125, -0.2013, -0.2125, -0.0253,
         -0.1226, -0.0478],
        [ 0.0119,  0.1710, -0.3109,  0.0423, -0.2316, -0.2336, -0.2374, -0.0209,
         -0.1320, -0.0563],
        [-0.0461,  0.1574, -0.2655, -0.0013, -0.2279, -0.1385, -0.1259, -0.0393,
         -0.1756, -0.0223],
        [-0.0240,  0.1733, -0.2430,  0.0173, -0.2228, -0.1888, -0.1855, -0.0224,
         -0.1417, -0.0553],
        [-0.0453,  0.1876, -0.2549,  0.0383, -0.2034, -0.1832, -0.1964, -0.0251,
         -0.1398, -0.0417]], grad_fn=<AddmmBackward0>)
pre => tensor([1, 1, 1, 1, 1])


In [27]:
for name,param in m.named_parameters():
    print(f'[{name}]-------- \n{param.shape}')

[in_layer.weight]-------- 
torch.Size([10, 3, 3, 3])
[in_layer.bias]-------- 
torch.Size([10])
[hd_layer.weight]-------- 
torch.Size([20, 2250])
[hd_layer.bias]-------- 
torch.Size([20])
[out_layer.weight]-------- 
torch.Size([10, 20])
[out_layer.bias]-------- 
torch.Size([10])
